<a href="https://colab.research.google.com/github/gabrielawad/talleresGoogleColab/blob/main/Taller_20_CPM_PERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## «Nadie es perfecto en este mundo imperfecto».
### [Patrice Lumumba](https://es.wikipedia.org/wiki/Patrice_Lumumba)

# Taller 20 CPM y PERT

En este taller se presentan los pasos para resolver ejercicios de CPM (ruta crítica) y PERT utilizando Python.

## Paso 0.0: Importar las librerías requeridas

In [ ]:
# Importar las librerías requeridas
import numpy as np
import pandas as pd

# Solución de un ejercicio de ruta crítica (CPM)

## Paso 1.0: obtener datos

Leer los datos del archivo y verificar que hayan sido adecuadamente guardados en un objeto de pandas.

**Los datos se encuentran en: [cpmEjemplo.xlsx](https://docs.google.com/spreadsheets/d/e/2PACX-1vSnt1r41grZTV3-15LBEBLgERXzBdViWC7QLu4lmS20M8adQMqBaoA3xEdpmgNd9Q/pub?output=xlsx)**

In [ ]:
# Leer la información del archivo y cargarla en un DataFrame
ruta_cpm ='https://docs.google.com/spreadsheets/d/e/2PACX-1vSnt1r41grZTV3-\
15LBEBLgERXzBdViWC7QLu4lmS20M8adQMqBaoA3xEdpmgNd9Q/pub?output=xlsx'

datosP =pd.read_excel(ruta_cpm)

#datosP =pd.read_excel('cpmEjemplo.xlsx')

# Verificar que el archivo fue adecuadamente leído y
# que los datos se encuentran disponibles
print ("Los contenidos de datosP son: \n", datosP)

## Paso 1.1: calcular la ruta crítica


In [ ]:
# Cantidad de actividades del proyecto
cantP = datosP.Duración.count()

# Suma de todos los tiempos de las actividades del proyecto
tmax = datosP.Duración.sum()

# Cantidad de eventos del proyecto
cant = datosP.Evento.max() + 1

# Encontrar los tiempos más próximos y más lejanos de cada evento
# Crear el arreglo datosE con una longitud igual al número de eventos del
# proyecto
datosE = np.zeros((cant, 3), dtype=int)

# Colocar los eventos en la primera columna de datosE
datosE[:,0] = range (cant)

# Convertir datosE en un DataFrame y agregar las correspondientes etiquetas
datosE = pd.DataFrame(datosE, columns=['Evento', 'iniTemprano', 'terTardia'])

# Crear etiquetas para las nuevas columnas requeridas en datosP
nombres = {'Holgura', 'Tpredecesor','Tsucesor'}

# Agregar las columnas requeridas datosP con todos sus valores iguales a cero
for i in nombres:
    datosP[i] = 0

# Encontrar los tiempos más tempranos de inicio de cada evento
for i in range(1,cant):
    w = datosP[datosP.Evento == i]
    mayor = np.max(w.Duración + w.Tpredecesor)
    datosP.loc[datosP.EvAnterior == i,'Tpredecesor']= mayor
    # Guardar en datosE el tiempo más cercano
    datosE.loc[i,'iniTemprano']= mayor

# Actualizar la información del último evento
datosE.loc[cant-1,'terTardia']= mayor
datosP.loc[datosP.Evento == cant-1,'Tsucesor']= mayor

# Encontrar los tiempos más lejanos de terminación de cada evento
for i in range(cant-2, -1, -1):
    z = datosP[datosP.EvAnterior == i]
    menor = np.min(z.Tsucesor - z.Duración)
    datosP.loc[datosP.Evento == i,'Tsucesor']= menor
    # Guardar en datosE el tiempo más lejano
    datosE.loc[i,'terTardia']= menor

# Encontrar los eventos que hacen parte de la ruta crítica
datosE['rutaCritica'] = datosE['terTardia']-datosE['iniTemprano']

# Convertir los datos de los eventos en un arreglo de NumPy
# pos 0: Evento
# pos 1: tiempo más próximo
# pos 2: tiempo más lejano
# pos 3: rutaCrítica
datosE1 = np.array (datosE)

# Construir el arreglo para el cálculo de holgura de las actividades
holg = np.array (datosP.loc[ datosP.Evento > 0,
 [ 'EvAnterior', 'Evento', 'Duración', 'Holgura']])

# Calcular las holguras de las actividades
for i in range(cantP-1):
    holg [i,3]= datosE1[holg[i,1],2] - (datosE1[holg[i,0],1] + holg[i,2])



## Paso 1.2: mostrar las respuestas

In [ ]:
# Mostrar la información general del proyecto
print('La cantidad de actividades del proyecto es:', cantP,"\n")
print('La suma de todos los tiempos de las actividades del proyecto es:',
      tmax,"\n")
print("El proyecto se compone de", cant, "eventos. \n")

# Mostrar los eventos que hacen parte de la ruta crítica
print ('Esta es la ruta crítica de eventos: \n')
for i in range (cant):
    if datosE1[i,3]==0:
        print(datosE1[i,0])

# Mostrar los eventos que no hacen parte de la ruta crítica
print ('\n Los eventos que no hacen parte de la ruta crítica son: \n')
for i in range (cant):
    if datosE1[i,3]!=0:
        print(datosE1[i,0])

# Mostrar la ruta crítica de las actividades
for i in range (cantP-1):
    if holg [i,3] ==0:
        a =str(holg [i,0])
        b = str(holg [i,1])
        print ("La actividad", a+","+b ,
               "pertenece a la ruta crítica y tiene una holgura de",
               holg [i,3],"\n")

# Mostrar las holguras de las actividades
for i in range (cantP-1):
    if holg [i,3] !=0:
        a =str(holg [i,0])
        b = str(holg [i,1])
        print ("La actividad", a+","+b ,
               "NO pertenece a la ruta crítica y tiene una holgura de",
               holg [i,3],"\n")

# Solución de un ejercicio de PERT

## Paso 2.0: obtener datos
Leer los datos del archivo y verificar que hayan sido adecuadamente guardados en un objeto de pandas.

**Los datos se encuentran en: [pertEjemplo.xlsx](https://docs.google.com/spreadsheets/d/e/2PACX-1vSshUQyh5SdMD-s0fH93Hs79OS-skErBi7t1xWksD2mDvC3IwFHQt4PXMvXZxr2BBLmlDTjNvpC35Kh/pub?output=xlsx)**

In [ ]:
# Leer la información del archivo y cargarla en un DataFrame
ruta_pert ='https://docs.google.com/spreadsheets/d/e/2PACX-1vS-6SJmw0r2vNXc3g7-\
6_b66SJrjG0UYWE-1BpbAoZZEi-cvUFng1Yiuq3UKfyoMw/pub?output=xlsx'

datosP =pd.read_excel(ruta_pert)

#datosP =pd.read_excel('pertEjemplo.xlsx')

# Verificar que el archivo fue adecuadamente leído y
# que los datos se encuentran disponibles
print ("Los contenidos de datosP son: \n", datosP)

## Paso 2.1: calcular la ruta crítica

In [ ]:
# Calcular la duración esperada de cada actividad
datosP['Duración'] = 1/3 *(2*datosP['m'] +  1/2 *(datosP['a'] + datosP['b']))

# Calcular la varianza de cada actividad
datosP['Varianza'] = (1/6 *(datosP['b'] - datosP['a']))**2

# datosP['Duración'] = 1
# Cantidad de actividades del proyecto
cantP = datosP.Duración.count()

# Suma de todos los tiempos de las actividades del proyecto
tmax = datosP.Duración.sum()

# Cantidad de eventos del proyecto
cant = datosP.Evento.max() + 1

# Encontrar los tiempos más próximos y más lejanos de cada evento
# Crear el arreglo datosE con una longitud igual al número de eventos del
# proyecto
datosE = np.zeros((cant, 3), dtype=int)

# Colocar los eventos en la primera columna de datosE
datosE[:,0] = range (cant)

# Convertir datosE en un DataFrame y agregar las correspondientes etiquetas
datosE = pd.DataFrame(datosE, columns=['Evento', 'iniTemprano', 'terTardia'])

# Crear etiquetas para las nuevas columnas requeridas en datosP
nombres = {'Holgura', 'Tpredecesor','Tsucesor'}

# Agregar las columnas requeridas datosP con todos sus valores iguales a cero
for i in nombres:
    datosP[i] = 0

# Encontrar los tiempos más tempranos de inicio de cada evento
for i in range(1,cant):
    w = datosP[datosP.Evento == i]
    mayor = np.max(w.Duración + w.Tpredecesor)
    datosP.loc[datosP.EvAnterior == i,'Tpredecesor']= mayor
    # Guardar en datosE el tiempo más cercano
    datosE.loc[i,'iniTemprano']= mayor

# Actualizar la información del último evento
datosE.loc[cant-1,'terTardia']= mayor
datosP.loc[datosP.Evento == cant-1,'Tsucesor']= mayor

# Encontrar los tiempos más lejanos de terminación de cada evento
for i in range(cant-2, -1, -1):
    z = datosP[datosP.EvAnterior == i]
    menor = np.min(z.Tsucesor - z.Duración)
    datosP.loc[datosP.Evento == i,'Tsucesor']= menor
    # Guardar en datosE el tiempo más lejano
    datosE.loc[i,'terTardia']= menor

# Encontrar los eventos que hacen parte de la ruta crítica
datosE['rutaCritica'] = datosE['terTardia']-datosE['iniTemprano']

# Convertir los datos de los eventos en un arreglo de NumPy
# pos 0: Evento
# pos 1: tiempo más próximo
# pos 2: tiempo más lejano
# pos 3: rutaCrítica
datosE1 = np.array (datosE)

# Construir el arreglo para el cálculo de holgura de las actividades
holg = np.array (datosP.loc[ datosP.Evento > 0, ['EvAnterior', 'Evento',
                                                 'Duración', 'Holgura']])

# Calcular las holguras de las actividades
for i in range(cantP-1):
    holg [i,3]= datosE1[holg[i,1],2] - (datosE1[holg[i,0],1] + holg[i,2])

## Paso 2.2: mostrar las respuestas

In [ ]:
# Mostrar la información general del proyecto
print('La cantidad de actividades del proyecto es:', cantP,"\n")
print('La suma de todos los tiempos de las actividades del proyecto es:',
      tmax,"\n")
print("El proyecto se compone de", cant, "eventos. \n")

# Mostrar los eventos que hacen parte de la ruta crítica
print ('Esta es la ruta crítica de eventos: \n')
for i in range (cant):
    if datosE1[i,3]==0:
        print(int (datosE1[i,0]))

# Mostrar los eventos que no hacen parte de la ruta crítica
print ('\n Los eventos que no hacen parte de la ruta crítica son: \n')
for i in range (cant):
    if datosE1[i,3]!=0:
        print(int(datosE1[i,0]))

# Mostrar la ruta crítica de las actividades
for i in range (cantP):
    if holg [i,3] ==0:
        a =str(holg [i,0])
        b = str(holg [i,1])
        print ("La actividad", a+","+b,
               "pertenece a la ruta crítica y tiene una holgura de",
               holg [i,3],"\n")

# Mostrar las holguras de las actividades
for i in range (cantP-1):
    if holg [i,3] !=0:
        a =str(holg [i,0])
        b = str(holg [i,1])
        print ("La actividad", a+","+b ,
               "NO pertenece a la ruta crítica y tiene una holgura de",
               holg [i,3],"\n")

# Ejercicios

Resuelva los ejercicios propuestos en el [Taller_08_CPM_PERT](https://docs.google.com/document/d/1iNb4Uo2RWthsaSdiLfKVeuLEoafBvAa3seGQOBl_fvc/edit?usp=sharing)

## Autopistas de la prosperidad

## Solución de conflictos armados